Clase 3(a) - Calculo de índices espectrales
===========================================

### SoPI II: Herramientas de teledetección cuantitativa

En esta clase nos centraremos en el calculo de índices espectrales y la obtención de variables biofísicas a partir de los mismos.

Se utilizara este notebook de python para ayudar en el procesamiento.

Los notebooks se dividen de la siguiente manera

- *(a)* - Estimación de la linea de suelo para una imagen.
- (b) - Grafica de la variación temporal de un índice.
- (c) - Descomposición en componentes de tendencia y variación temporal.

### Carga de librerias

Recuerde que debe ejecutar **siempre** primero la celda que carga las librerias para ejecutar los procesos.

Durante esta clase utilizaremos las librerias

- [matplotlib](http://matplotlib.org/) para generación de gráficos.
- [numpy](http://www.numpy.org/) para el procesamiento numérico y matricial.
- [scipy](http://www.scipy.org) para realizar los ajustees lineales.
- [gdal](http://www.gdal.org) para la apertura de imágenes.

In [8]:
# Cargamos las librerais
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import gdal

# Fijamos que muestre los graficos dentro de la linea
%matplotlib inline

Seleccionamos la imagen sobre la cual queremos calcular la linea de tendencia

In [3]:
# Nombre de la imagen
imagen_in = "../../material/imagenes/l8_oli_20130819.tif"

A continuación cargamos la imagen y sus propiedades (número de filas, columnas, etc).

Luego de esto leemos las bandas del rojo y el infrarrojo cercano como array de numpy y lo convertimos en un array lineal.

In [241]:
# cargo la imagen como un handle
imagen_ha = gdal.Open(imagen_in)

# Leo la cantidad de filas y columnas
Nx = imagen_ha.RasterXSize
Ny = imagen_ha.RasterYSize
N  = imagen_ha.RasterCount


# Leo las bandas 3 y 4 a dos arrays
red = imagen_ha.GetRasterBand(3).ReadAsArray()
nir = imagen_ha.GetRasterBand(4).ReadAsArray()

# Convierto los arrays a 1D
red.shape = Nx*Ny
nir.shape = Nx*Ny

Para encontrar la linea de suelo, buscamos ajustar los valores de brillo de la imagen que se encuentren en el percentil mas bajo.

Además, descartaremos los valores que esten por debajo de de cierto valor de brillo de corte para evitar ajustar la imagen por puntos provenientes de cuerpos de agua

In [341]:
limi = (red>900)&(red<3500)
r1=red[limi]
n1=nir[limi]
ratio = n1/r1

In [ ]:
mask = (ratio < np.percentile(ratio,0.2))&(ratio > np.percentile(ratio,0.07))
L = stats.linregress(r1[mask],n1[mask])
x = np.linspace(0,4000,10)
y = L[0]*x+L[1]
plt.scatter(red[(red>1000)],nir[(red>1000)])
plt.scatter(r1[mask],n1[mask],color="green")
plt.plot(x,y,color="red",linewidth=2)

In [339]:
L

LinregressResult(slope=0.96922283063186498, intercept=24.790471909625694, rvalue=0.96456921942612384, pvalue=1.7162725127695438e-174, stderr=0.015356991097447537)